Setup

In [1]:
from pathlib import Path
import pandas as pd

SITE_ID = "MAO"
ZONE    = "DORM1"

DATA_DIR   = Path("../data")
INTERIM    = DATA_DIR / "interim" / SITE_ID
PROCESSED  = DATA_DIR / "processed" / SITE_ID
PROCESSED.mkdir(parents=True, exist_ok=True)

# arquivos de entrada
F_MET   = INTERIM / "met_aligned.csv.gz"
F_VN    = INTERIM / "eplus_vn.csv.gz"
F_AC    = INTERIM / "eplus_ac.csv.gz"   # se existir

# JOS3: agora são vários (Caso1/2/3)
JOS3_PATTERN = f"jos3_output_{ZONE}_Caso*_vn.csv.gz"
F_JOS3_LIST  = sorted(PROCESSED.glob(JOS3_PATTERN))

if not F_JOS3_LIST:
    raise FileNotFoundError(f"Nenhum arquivo JOS3 encontrado em {PROCESSED} com padrão: {JOS3_PATTERN}")

# saída
OUT_BASE = PROCESSED / f"{SITE_ID}_{ZONE}_1991-2023_FULL_PIPELINE"

Helpers

In [2]:
def read_csv_timeset(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path, compression="gzip", low_memory=False)
    # garantir timeset como datetime e remover duplicatas
    if "timeset" not in df.columns:
        raise ValueError(f"{path.name} não possui coluna 'timeset'.")
    df["timeset"] = pd.to_datetime(df["timeset"], errors="coerce")
    df = df.dropna(subset=["timeset"]).sort_values("timeset")
    df = df[~df["timeset"].duplicated(keep="first")]
    return df.reset_index(drop=True)

def pref(df: pd.DataFrame, prefix: str, keep=("timeset",)) -> pd.DataFrame:
    """Prefixa todas as colunas exceto as de `keep`."""
    ren = {c: (c if c in keep else f"{prefix}{c}") for c in df.columns}
    return df.rename(columns=ren)

In [3]:
def read_csv_timeset_by(path: Path, by=("scenario",)) -> pd.DataFrame:
    df = pd.read_csv(path, compression="gzip", low_memory=False)
    if "timeset" not in df.columns:
        raise ValueError(f"{path.name} não possui coluna 'timeset'.")
    df["timeset"] = pd.to_datetime(df["timeset"], errors="coerce")
    df = df.dropna(subset=["timeset"]).sort_values("timeset")

    key = [c for c in (*by, "timeset") if c in df.columns]
    if key:
        df = df.drop_duplicates(subset=key, keep="first")

    return df.reset_index(drop=True)

Carregar bases

In [4]:
# bases temporais
met = read_csv_timeset(F_MET)       # EPW + flags + métricas
vn  = read_csv_timeset_by(F_VN, by=("scenario",))
ac  = read_csv_timeset_by(F_AC, by=("scenario",))      

# JOS3: Caso1/2/3
jos3_list = []

for f in F_JOS3_LIST:
    df = read_csv_timeset_by(f, by=("scenario",))  # <- aqui

    if "scenario" not in df.columns:
        scenario = f.stem.replace(f"jos3_output_{ZONE}_", "")
        df["scenario"] = scenario

    jos3_list.append(df)

jos3 = pd.concat(jos3_list, ignore_index=True)

# garantir unicidade por scenario+timeset no conjunto final
jos3 = (
    jos3.sort_values(["scenario", "timeset"])
        .drop_duplicates(subset=["scenario", "timeset"], keep="first")
        .reset_index(drop=True)
)

In [5]:
print("met :",  met["timeset"].min(),  "→", met["timeset"].max(),  "len:", len(met))
print("vn  :",  vn["timeset"].min(),   "→", vn["timeset"].max(),   "len:", len(vn))
print("jos3:",  jos3["timeset"].min(), "→", jos3["timeset"].max(), "len:", len(jos3))
print("ac  :",  ac["timeset"].min(),   "→", ac["timeset"].max(),   "len:", len(ac))

print("\nCenários VN:", vn["scenario"].unique())
print("\nCenários AC:", ac["scenario"].unique())
print("\nCenários JOS3:", jos3["scenario"].unique())

met : 1991-01-01 01:00:00 → 2024-01-01 00:00:00 len: 289080
vn  : 1991-01-01 01:00:00 → 2024-01-01 00:00:00 len: 867240
jos3: 1991-01-01 01:00:00 → 2024-01-01 00:00:00 len: 867240
ac  : 1991-01-01 01:00:00 → 2024-01-01 00:00:00 len: 867240

Cenários VN: ['Caso1_vn' 'Caso2_vn' 'Caso3_vn']

Cenários AC: ['Caso1_ac' 'Caso2_ac' 'Caso3_ac']

Cenários JOS3: ['Caso1_vn' 'Caso2_vn' 'Caso3_vn']


Prefixar e unir

In [6]:
# Prefixar, mas manter timeset e scenario sem prefixo
vn_p   = pref(vn,   "vn_",   keep=("timeset","scenario"))
jos3_p = pref(jos3, "jos3_", keep=("timeset","scenario"))
ac_p   = pref(ac,   "ac_",   keep=("timeset","scenario"))

# Base: met replicado por cenário (a partir dos cenários do VN)
scenarios = sorted(vn["scenario"].unique())
met_rep = met.merge(pd.DataFrame({"scenario": scenarios}), how="cross")

# Merge sempre por scenario+timeset
full = met_rep.merge(vn_p,   on=["scenario","timeset"], how="left")
full = full.merge(jos3_p,    on=["scenario","timeset"], how="left")

# AC: seus cenários são Caso*_ac, enquanto VN/JOS3 são Caso*_vn.
# Precisamos mapear o "case" para alinhar VN<->AC.
# Cria uma coluna "case" canônica (Caso1, Caso2, Caso3) em ambos.
full["case"] = full["scenario"].str.replace(r"_(vn|ac)$", "", regex=True)

ac_p2 = ac_p.copy()
ac_p2["case"] = ac_p2["scenario"].str.replace(r"_(vn|ac)$", "", regex=True)

# Faz merge do AC por case + timeset (não por scenario, pois sufixo difere)
full = full.merge(ac_p2.drop(columns=["scenario"]), on=["case","timeset"], how="left")

full = full.sort_values(["scenario","timeset"]).reset_index(drop=True)

print("full:", full["timeset"].min(), "→", full["timeset"].max(), "len:", len(full))
print("dups (scenario,timeset):", full.duplicated(subset=["scenario","timeset"]).sum())
print("scenarios in full:", full["scenario"].nunique())

full: 1991-01-01 01:00:00 → 2024-01-01 00:00:00 len: 867240
dups (scenario,timeset): 0
scenarios in full: 3


QC básico

In [7]:
# NaNs por bloco (amostra)
cols_flags = [c for c in full.columns if c.startswith(("HW_","INMET_","OUZ_"))]
cols_vn    = [c for c in full.columns if c.startswith("vn_")]
cols_jos3  = [c for c in full.columns if c.startswith("jos3_")]
cols_ac    = [c for c in full.columns if c.startswith("ac_")]

def nan_ratio(df, cols, label):
    if cols:
        print(label, "NaN:", df[cols].isna().mean().mean().round(3))

print("\n=== QC GLOBAL (todos os cenários juntos) ===")
nan_ratio(full, cols_flags, "Flags")
nan_ratio(full, cols_vn,    "VN")
nan_ratio(full, cols_jos3,  "JOS3")
nan_ratio(full, cols_ac,    "AC")

print("\n=== QC POR CENÁRIO ===")
for scen, dfg in full.groupby("scenario"):
    print(f"\n[{scen}]")
    nan_ratio(dfg, cols_flags, "Flags")
    nan_ratio(dfg, cols_vn,    "VN")
    nan_ratio(dfg, cols_jos3,  "JOS3")
    nan_ratio(dfg, cols_ac,    "AC")


=== QC GLOBAL (todos os cenários juntos) ===
Flags NaN: 0.763
VN NaN: 0.0
JOS3 NaN: 0.0
AC NaN: 0.0

=== QC POR CENÁRIO ===

[Caso1_vn]
Flags NaN: 0.763
VN NaN: 0.0
JOS3 NaN: 0.0
AC NaN: 0.0

[Caso2_vn]
Flags NaN: 0.763
VN NaN: 0.0
JOS3 NaN: 0.0
AC NaN: 0.0

[Caso3_vn]
Flags NaN: 0.763
VN NaN: 0.0
JOS3 NaN: 0.0
AC NaN: 0.0


In [8]:
def valid_hours(df, cols):
    return 1 - df[cols].isna().mean()

print("\n=== % horas válidas JOS3 por cenário ===")
for scen, dfg in full.groupby("scenario"):
    vh = valid_hours(dfg, cols_jos3).mean()
    print(f"{scen}: {vh:.2%}")


=== % horas válidas JOS3 por cenário ===
Caso1_vn: 100.00%
Caso2_vn: 100.00%
Caso3_vn: 100.00%


Salvar

In [9]:
full.to_csv(f"{OUT_BASE}.csv.gz", index=False, compression="gzip")
print("Saved:", f"{OUT_BASE}.csv.gz")

Saved: ..\data\processed\MAO\MAO_DORM1_1991-2023_FULL_PIPELINE.csv.gz


In [10]:
full.to_csv(f"{OUT_BASE}.csv", index=False)
print("Saved:", f"{OUT_BASE}.csv")

Saved: ..\data\processed\MAO\MAO_DORM1_1991-2023_FULL_PIPELINE.csv


In [11]:
full.to_parquet(f"{OUT_BASE}.parquet", index=False)
print("Saved:", f"{OUT_BASE}.parquet")

Saved: ..\data\processed\MAO\MAO_DORM1_1991-2023_FULL_PIPELINE.parquet
